## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [28]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Santa Isabel,BR,-23.3156,-46.2214,71.01,73,0,0.63,clear sky
1,1,Geraldton,AU,-28.7667,114.6000,73.40,64,90,14.97,overcast clouds
2,2,Bambous Virieux,MU,-20.3428,57.7575,79.00,78,75,11.50,broken clouds
3,3,Tahoua,NE,14.8888,5.2692,85.91,6,46,10.69,scattered clouds
4,4,Vaini,TO,-21.2000,-175.2000,80.60,74,20,9.22,few clouds


In [29]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 65


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Bredasdorp,ZA,-34.5322,20.0403,53.60,100,100,0.67,overcast clouds
10,10,Ushuaia,AR,-54.8000,-68.3000,50.00,57,40,5.75,scattered clouds
15,15,Pacific Grove,US,36.6177,-121.9166,59.00,58,1,10.36,clear sky
19,19,Bluff,NZ,-46.6000,168.3333,60.01,87,88,10.00,light rain
20,20,Dinghai,CN,26.2847,119.7936,60.80,88,75,4.47,broken clouds
24,24,Saldanha,ZA,-33.0117,17.9442,60.80,88,100,9.66,overcast clouds
27,27,Hasaki,JP,35.7333,140.8333,54.00,66,20,8.05,few clouds
34,34,Hermanus,ZA,-34.4187,19.2345,59.00,93,100,2.95,overcast clouds
36,36,Mataura,NZ,-46.1927,168.8643,60.01,87,69,10.00,light rain
51,51,Muros,ES,42.7762,-9.0603,53.01,81,99,14.88,overcast clouds


In [31]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

City_ID                148
City                   148
Country                148
Lat                    148
Lng                    148
Max Temp               148
Humidity               148
Cloudiness             148
Wind Speed             148
Current Description    148
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis = 0)

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Bredasdorp,ZA,53.60,overcast clouds,-34.5322,20.0403,
10,Ushuaia,AR,50.00,scattered clouds,-54.8000,-68.3000,
15,Pacific Grove,US,59.00,clear sky,36.6177,-121.9166,
19,Bluff,NZ,60.01,light rain,-46.6000,168.3333,
20,Dinghai,CN,60.80,broken clouds,26.2847,119.7936,
24,Saldanha,ZA,60.80,overcast clouds,-33.0117,17.9442,
27,Hasaki,JP,54.00,few clouds,35.7333,140.8333,
34,Hermanus,ZA,59.00,overcast clouds,-34.4187,19.2345,
36,Mataura,NZ,60.01,light rain,-46.1927,168.8643,
51,Muros,ES,53.01,overcast clouds,42.7762,-9.0603,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    long = row["Lng"]

    params["location"] = f"{lat},{long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Bredasdorp,ZA,53.60,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
10,Ushuaia,AR,50.00,scattered clouds,-54.8000,-68.3000,Albatros Hotel
15,Pacific Grove,US,59.00,clear sky,36.6177,-121.9166,Lovers Point Inn
19,Bluff,NZ,60.01,light rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
20,Dinghai,CN,60.80,broken clouds,26.2847,119.7936,


In [66]:
hotel_clean_df = hotel_df.dropna(how='all',axis=0)
hotel_clean_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Bredasdorp,ZA,53.60,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
10,Ushuaia,AR,50.00,scattered clouds,-54.8000,-68.3000,Albatros Hotel
15,Pacific Grove,US,59.00,clear sky,36.6177,-121.9166,Lovers Point Inn
19,Bluff,NZ,60.01,light rain,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
20,Dinghai,CN,60.80,broken clouds,26.2847,119.7936,
24,Saldanha,ZA,60.80,overcast clouds,-33.0117,17.9442,Blue Bay Lodge
27,Hasaki,JP,54.00,few clouds,35.7333,140.8333,Hotel Sunrise Choshi
34,Hermanus,ZA,59.00,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
36,Mataura,NZ,60.01,light rain,-46.1927,168.8643,Ellie's Villa
51,Muros,ES,53.01,overcast clouds,42.7762,-9.0603,Apartment Av. Castelao


In [67]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [68]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [73]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer_hotels = gmaps.heatmap_layer(locations, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer_hotels)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))